In [1]:
import pandas as pd
import Levenshtein
import itertools
from fuzzywuzzy import fuzz, process
import ast
from selenium import webdriver
import time
import webbrowser

In [2]:
recipe_df = pd.read_csv('../data/processed/ingredients_df.csv')

pd.set_option('max_columns', None)

recipe_df.head()

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,variable,ingredient
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevo', 'jas verde...",60.0,0,0,0,0,0,principal,12,0,patatas pequeñas rojas
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,0,0,0,0,0,principal,9,0,foie micuit
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,0,0,0,0,0,principal,16,0,bacalao desalado
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,0,0,0,0,0,principal,12,0,langostinos
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,0,0,0,0,0,principal,16,0,pechugas de pollo


In [3]:
input_list = ['huevos', 'pechiga de pollo', 'lentejas', 'pimiento', 'arroz', 'leche', 'mantequilla',
              'harina', 'sal', 'azúcar', 'aceite', 'zanahorias', 'queso', 'bacon', 'lechuga', 'calabacín',
              'patatas', 'pollo', 'atún', 'garbanzos', 'judías verdes', 'jamón', 'jamón cocido', 'pan', 
              'pasta']

In [4]:
def fuzzy_columns(column, input_string):
    ratio = fuzz.WRatio(column, input_string)
    return ratio

In [5]:
for ingred in input_list:
    recipe_df[ingred] = recipe_df.apply(lambda x: fuzzy_columns(x["ingredient"], ingred), axis=1)

In [6]:
recipe_df

,recipe_image_url,recipe_name,recipe_url,ingredients_list,time_preparation(min),gluten_free,egg_free,sucrose_fructose_free,low_sodium,lactose_free,main_category,num_ingredients,variable,ingredient,huevos,pechiga de pollo,lentejas,pimiento,arroz,leche,mantequilla,harina,sal,azúcar,aceite,zanahorias,queso,bacon,lechuga,calabacín,patatas,pollo,atún,garbanzos,judías verdes,jamón,jamón cocido,pan,pasta
0,https://content-recetas.lecturas.com/medio/201...,Ensalada niçoise,https://www.lecturas.com/recetas/ensalada-nico...,"['patatas pequeñas rojas', 'huevo', 'jas verde...",60.0,0,0,0,0,0,principal,12,0,patatas pequeñas rojas,45,32,34,22,36,36,40,30,30,36,30,36,72,18,39,34,90,18,60,40,45,45,24,60,72
1,https://content-recetas.lecturas.com/medio/201...,Ensalada del périgord,https://www.lecturas.com/recetas/ensalada-peri...,"['foie micuit', 'beicon gruesas', 'champiñones...",25.0,0,0,0,0,0,principal,9,0,foie micuit,15,28,21,42,12,25,27,15,0,18,45,19,18,18,39,11,13,18,30,10,9,22,34,0,18
2,https://content-recetas.lecturas.com/medio/202...,Xató con salsa romesco,https://www.lecturas.com/recetas/xato-salsa-ro...,"['bacalao desalado', 'cebolleta', 'escarolas',...",35.0,0,0,0,0,0,principal,16,0,bacalao desalado,30,44,34,22,36,19,30,30,90,54,30,36,36,54,26,45,39,36,30,40,36,22,37,30,36
3,https://content-recetas.lecturas.com/medio/201...,Ensalada de ceviche de langostinos y cebolla,https://www.lecturas.com/recetas/ensalada-cevi...,"['langostinos', 'pimienta', 'limas', 'tomates'...",25.0,0,0,0,0,0,principal,12,0,langostinos,30,22,42,32,36,18,36,35,30,18,30,48,36,38,26,32,33,25,60,40,26,45,45,60,38
4,https://content-recetas.lecturas.com/medio/201...,"Ensalada de pollo, remolacha y manzana",https://www.lecturas.com/recetas/ensalada-poll...,"['pechugas de pollo', 'manzanas ácida', 'col l...",40.0,0,0,0,0,0,principal,16,0,pechugas de pollo,45,91,34,24,34,54,32,30,30,18,30,26,36,18,77,24,26,90,30,30,41,22,32,30,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174669,https://t1.rg.ltmcdn.com/es/images/4/4/6/img_p...,"Pavo relleno con castaÑas, higados de pollo y ...",https://www.recetasgratis.net/receta-de-pavo-r...,"['ingredientes ', 'a kilos', 'o limones partid...",NaN,0,0,0,0,0,principal,39,38,la pulpa de una manzana,15,34,34,34,36,18,38,45,30,36,15,43,18,36,39,43,39,36,60,40,38,45,24,60,36
174670,https://t2.rg.ltmcdn.com/es/images/9/1/9/img_l...,Langostinos arco iris,https://www.recetasgratis.net/receta-de-langos...,"['langostinos, tipo marfil,', 'mantequilla', '...",NaN,0,0,0,0,0,principal,39,38,azúcar.,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0,30,36,24,22,36,30,40
174671,https://t2.rg.ltmcdn.com/es/images/5/4/6/img_p...,Pavo relleno con arroz salvaje,https://www.recetasgratis.net/receta-de-pavo-r...,"['a kilos', 'limones', 'cebolla', 'vinagre', '...",480.0,0,0,0,0,0,principal,42,39,pollo,18,90,15,31,20,20,36,0,30,0,0,18,20,20,17,18,17,100,0,18,0,0,36,30,20
174672,https://t2.rg.ltmcdn.com/es/images/5/4/6/img_p...,Pavo relleno con arroz salvaje,https://www.recetasgratis.net/receta-de-pavo-r...,"['a kilos', 'limones', 'cebolla', 'vinagre', '...",480.0,0,0,0,0,0,principal,42,40,azúcar,0,19,18,0,40,20,25,36,30,100,36,40,0,40,33,36,33,0,30,36,24,22,36,30,40


In [7]:
input_list

['huevos',
 'pechiga de pollo',
 'lentejas',
 'pimiento',
 'arroz',
 'leche',
 'mantequilla',
 'harina',
 'sal',
 'azúcar',
 'aceite',
 'zanahorias',
 'queso',
 'bacon',
 'lechuga',
 'calabacín',
 'patatas',
 'pollo',
 'atún',
 'garbanzos',
 'judías verdes',
 'jamón',
 'jamón cocido',
 'pan',
 'pasta']

In [8]:
filter_huevos = recipe_df['huevos'] > 86

recipe_df[filter_huevos]["ingredient"].value_counts().index.tolist()


['huevo', 'huevos', 'huevos cocidos pelados y picados en rodajas']

In [9]:
filter_pechiga_de_pollo = recipe_df['pechiga de pollo'] > 86
recipe_df[filter_pechiga_de_pollo]["ingredient"].value_counts()

pollo                1018
pechugas de pollo      88
pollo                  39
pollo.                 30
pechuga de pollo       18
/pollo                  3
- pollo.                3
pollo)                  3
pollo...)               1
pollo:                  1
pollo,                  1
- pollo                 1
pollo                   1
Name: ingredient, dtype: int64

In [10]:
filter_lentejas = recipe_df['lentejas'] > 86
recipe_df[filter_lentejas]["ingredient"].value_counts()

lentejas                             84
lentejas cocidas                     33
lentejas pardinas                    15
lentejas pardinas cocidas             8
lentejas cocinadas                    8
jas                                   7
lentejas                              6
lentejas remojadas                    4
lentejas secas                        4
lenteja                               3
té                                    2
lentejas cocidas                      2
gr lentejas                           2
lentejas rojas                        2
entre lentejas pardinas               1
lentejas guisadas o sólo hervidas     1
hummus de lentejas rojas              1
ñentejas                              1
lentejas cocinadas.                   1
lentejas.                             1
lentejas)                             1
jas                                   1
lentejas previamente hidratadas.      1
lentejas rojas lavados                1
grs lentejas                          1


In [11]:
filter_pimiento = recipe_df['pimiento'] > 86
recipe_df[filter_pimiento]["ingredient"].value_counts().index.tolist()

['pimienta',
 'pimiento rojo',
 'pimiento verde',
 'pimienta.',
 'pimientos rojos',
 'pimientos verdes',
 'pimiento amarillo',
 'pimiento',
 'pimiento morrón',
 'pimientos del piquillo',
 'pimiento del piquillo',
 'pimiento verde italiano',
 'pimientos morrones',
 'pimiento rojo asado',
 'pimiento rojo pequeño',
 'pimiento choricero',
 'pimientos',
 'pimienta ',
 'pimiento o pimentón rojo',
 'pimientos choriceros',
 'pimientos amarillos',
 'pimiento o pimentón',
 'pimenton',
 'pimiento verde.',
 'pimientos verdes italianos',
 'pimientos rojos asados',
 'pimientos rojos asado',
 '- pimienta.',
 'pimiento verde pequeño',
 'pimientos asados',
 'pimiento morrón rojo',
 '/pimiento rojo',
 'pimiento verde ',
 'pimiento rojo grande',
 'pimiento ',
 'pimiento rojo picado',
 'pimiento amarillo pequeño',
 'pimientos ',
 'pimiento morrón verde',
 'pimiento morrón asado',
 'un pimiento verde',
 'pimiento rojo y verde',
 'pimiento del piquillo asados',
 'pimientos rojos ',
 'pimiento rojo o verde',

In [12]:
filter_arroz = recipe_df['arroz'] > 86
recipe_df[filter_arroz]["ingredient"].value_counts().index.tolist()

['arroz',
 'arroz basmati',
 'arroz blanco',
 'arroz redondo',
 'arroz integral',
 'arroz ',
 'arroz largo',
 'arroz bomba',
 'arroz cocido',
 'arroz arborio',
 'arroz blanco cocido',
 'arroz blanco cocinado',
 'arroz cocido ',
 'harina de arroz',
 'arroz salvaje',
 'vinagre de arroz',
 'arroz integral ',
 'leche de arroz',
 'arroz.',
 'arroz carnaroli',
 'río',
 'arroz inflado de chocolate',
 'arroz integral cocido',
 'arroz jazmín',
 'arroz)',
 'arroz cr',
 'fideos de arroz',
 'arroz blanco ',
 'sirope de arroz',
 'g arroz',
 'oblea de arroz',
 'arroz nerone (venere)',
 'arroz arborio o carnaroli',
 'morcilla de arroz',
 'arroz hervido',
 'arroz integral cocinado',
 'arroz arboreo',
 'arroz o mirin',
 'oz',
 'arroz inflado',
 'arroz basmati cocido',
 'arroz integral.',
 'arroz pequeñas',
 'pasta de arroz',
 'arroz arroz cocido',
 'arroz blanco.',
 'arroz cocido sin sal',
 'arroz bomba o largo seco',
 'copos de arroz',
 'arroz previamente cocido',
 'kg arroz ',
 'arroz cocinado ',
 '-

In [13]:
filter_leche = recipe_df['leche'] > 86
recipe_df[filter_leche]["ingredient"].value_counts().index.tolist()

['leche',
 'leche ',
 'leche condensada',
 'leche entera',
 'leche evaporada',
 'leche o nata',
 'leche desnatada',
 'chocolate con leche',
 'leche vegetal',
 'leche de coco',
 'leche en polvo',
 'leche de soja',
 'leche tibia',
 'leche.',
 'chocolate con leche fondant',
 'leche)',
 'crema de leche',
 'leche condensada ',
 'leche o nata líquida',
 'dulce de leche',
 'gr leche',
 'leche semidesnatada',
 'leche de almendras',
 '¾ de leche ',
 'leche de arroz',
 'leche líquida',
 'ml leche',
 'leche caliente',
 'leche entera.',
 'leche de avena',
 'leche fría',
 'leche templada',
 'leche fresca',
 'leche o nata ',
 'leche descremada',
 '- leche.',
 'leche vegetal ',
 'crema de leche.',
 'leche o agua',
 'leche o nata.',
 'leche espesa',
 'l. leche',
 'leche o leche evaporada',
 'ml leche entera',
 'medio litro de leche',
 '/l. leche',
 'leche evaporada ideal',
 'leche evaporada ',
 'leche condensada  bote pequeño',
 'leche hervida',
 'agua o leche',
 'nata o leche evaporada',
 'leche vege

In [14]:
filter_mantequilla = recipe_df['mantequilla'] > 86
recipe_df[filter_mantequilla]["ingredient"].value_counts().index.tolist()

['mantequilla',
 'mantequilla sin sal',
 'mantequilla ',
 'mantequilla.',
 'mantequilla derretida',
 'g mantequilla',
 'mantequilla o margarina',
 'gr mantequilla',
 'maní',
 'gr. mantequilla',
 'mantequilla blanda',
 'mantequilla fundida',
 'mantequilla con sal',
 'mantequilla una nuez',
 'margarina o mantequilla',
 'mantequilla reblandecida',
 'manteca o mantequilla',
 'aceite o mantequilla',
 'mantequilla pomada',
 'mantequilla fría',
 'mantequilla clarificada',
 'mantequilla o aceite',
 'mantequilla ablandada',
 'mantequilla salada',
 'mantequilla de cacahuete',
 'mantequilla vegetal',
 'mantequilla al gusto',
 'mantequilla nuez',
 'matequilla',
 'cucharada mantequilla',
 'mantequilla o aceite vegetal',
 'gramos mantequilla',
 'mantequilla fresca',
 'mantequilla en pomada',
 'mantequilla fría sin sal',
 'mantequilla blanda.',
 'mantequilla, ',
 'mantequilla,',
 'mantequilla light',
 'mantequilla vegetal sin sal',
 'mantequilla sin sal derretida',
 '- mantequilla.',
 'cucharadita ma

In [15]:
filter_harina = recipe_df['harina'] > 90
recipe_df[filter_harina]["ingredient"].value_counts()

harina       3035
harina        377
harina.        25
g harina       13
/harina         3
hariana         2
harina .        2
- harina        1
y harina        1
- harina        1
harina,         1
Name: ingredient, dtype: int64

In [16]:
filter_sal = recipe_df['sal'] > 90
recipe_df[filter_sal]["ingredient"].value_counts()

sal       12045
sal.        243
sal          49
- sal.       14
- sal         3
•sal          3
•sal.         2
sal,          2
-sal .        1
.% sal        1
sal)          1
-sal          1
-sal.         1
sal ,         1
sal,          1
sal .         1
Name: ingredient, dtype: int64

In [17]:
filter_azúcar = recipe_df['azúcar'] > 90
recipe_df[filter_azúcar]["ingredient"].value_counts()

azúcar       3130
azúcar        647
azucar         58
azúcar.        19
azùcar          7
g azúcar        6
azucar.         3
•azúcar         3
azucar          3
azùcar.         2
azúcar,         2
/azúcar         2
-azúcar .       1
- azucar        1
/azucar         1
-azucar.        1
- azúcar        1
Name: ingredient, dtype: int64

In [18]:
filter_aceite = recipe_df['aceite'] > 86
recipe_df[filter_aceite]["ingredient"].value_counts().index.tolist()

['aceite de oliva',
 'aceite',
 'aceite ',
 'aceite vegetal',
 'aceite de oliva virgen extra',
 'aceite de girasol',
 'aceite virgen extra hojiblanca',
 'aceite de oliva virgen',
 'aceite de oliva suave',
 'atún en aceite',
 'aceite oliva virgen extra',
 'aceite de oliva.',
 'aceite oliva',
 'aceite.',
 'aceite de oliva ',
 'anchoas en aceite',
 'atún en conserva en aceite',
 'aceite neutro',
 'aceite de girasol ',
 'tomate seco en aceite',
 'atún en conserva en aceite de oliva',
 'tomates secos en aceite',
 'filetes de anchoa en aceite',
 'ml aceite',
 'aceite de coco',
 'anchoa en aceite',
 'aceite virgen extra',
 'caballa en aceite',
 'bonito en aceite',
 'aceite suave',
 'aceite o mantequilla',
 'gr aceite',
 'aceite y sal',
 'aceite girasol',
 'aceite vegetal ',
 'mantequilla o aceite',
 'aceite de sésamo',
 'aceite en aerosol',
 'aceite el necesario',
 'cch. aceite',
 'medio vaso de aceite de oliva',
 'aceite suave ',
 'aceite de oliva virgen extra ',
 'sardina en aceite',
 'g ac

In [19]:
filter_zanahorias = recipe_df['zanahorias'] > 86
recipe_df[filter_zanahorias]["ingredient"].value_counts().index.tolist()

['zanahoria',
 'zanahorias',
 'zanahorias.',
 'zanahorias pequeñas',
 'zanahoria.',
 'zanahorias baby',
 'zanahoria ',
 'zanahorias grandes',
 '/zanahoria',
 'zanahorias rallada',
 'zanahorias picadas',
 'zanahorias ralladas',
 'zanahorias en rodajas',
 'zanahorias cocidas',
 'zanahorias ',
 'tres zanahorias',
 '-zanahoria',
 'zanahorias en cubos',
 'zanahorias partidas en ruedas ',
 'guisantes y zanahorias baby.',
 'zanahorias limpias y troceadas',
 'zanhorias',
 'zanahorias picadas en cuadritos',
 'zanahorias chorizos',
 'zanahorias tiernas baby',
 'zanahorias, peladas, en rebanadas',
 'a zanahorias',
 'zanahorias crs ralladas',
 'calabacín sirve cualquier resto de verduras cocidas (acelgas puerros zanahorias)',
 'kg zanahorias',
 'zanahorias picaditas',
 'zanahorias ralladas.',
 'zanahorias tiernas en pequeños bastoncitos',
 'g. zanahorias',
 'conservas de verduras guisantes y zanahorias',
 'zanorias',
 'zanahorias grandes ralladas en tiritas',
 'fondant en forma de zanahorias',
 'z

In [20]:
filter_queso = recipe_df['queso'] > 86
recipe_df[filter_queso]["ingredient"].value_counts()

queso rallado                   468
queso parmesano                 343
queso blanco                    264
queso mozzarella                241
queso parmesano rallado         207
                               ... 
queso holandés rallado            1
queso mozzarella en lonchas       1
mayonesa o queso crema            1
queso mozzarella en rodajas       1
queso semiduro rallado            1
Name: ingredient, Length: 685, dtype: int64

In [21]:
filter_bacon = recipe_df['bacon'] > 86
recipe_df[filter_bacon]["ingredient"].value_counts()

bacon                                   123
bacon ahumado                            20
bacon o tocineta                         14
tocineta o bacon                         10
bacón                                     9
                                       ... 
bacon gr                                  1
bacon en dados                            1
bacon o panceta ahumada (sin lactosa      1
bacon cortado a lonchas                   1
bacon sin gordura                         1
Name: ingredient, Length: 70, dtype: int64

In [22]:
filter_lechuga = recipe_df['lechuga'] > 86
recipe_df[filter_lechuga]["ingredient"].value_counts()

lechuga                                         325
lechuga romana                                   41
lechugas                                         37
cogollo de lechuga                               18
lechuga iceberg                                  15
                                               ... 
lechuga de brotes variados                        1
lechuga  bolsa de hojas de ensalada variadas      1
lechuga hoja de roble cortadas                    1
lechugas de hoja de roble                         1
una lechuga romana grande                         1
Name: ingredient, Length: 128, dtype: int64

In [23]:
filter_calabacín = recipe_df['calabacín'] > 80
recipe_df[filter_calabacín]["ingredient"].value_counts()

calabacín                        487
calabacines                       67
calabacínes                       55
calabacín verde                   36
calabacín pequeño                 23
                                ... 
un calabacín                       1
calabacínes troceados              1
calabacínes pequenos o grande      1
flor de calabacín                  1
calabacín torneado                 1
Name: ingredient, Length: 65, dtype: int64

In [24]:
filter_patatas = recipe_df['patatas'] > 86
recipe_df[filter_patatas]["ingredient"].value_counts()

patatas                          818
patata                           395
patatas pequeñas                  53
patatas grandes                   52
patatas.                          24
                                ... 
patatas gorditas.                  1
patatas fritas finas               1
patatas en cuadros cocinadas.      1
patatas panaderas                  1
patatas crs                        1
Name: ingredient, Length: 132, dtype: int64

In [25]:
filter_pollo = recipe_df['pollo'] > 86
recipe_df[filter_pollo]["ingredient"].value_counts()

pollo                                1018
pechugas de pollo                      88
caldo de pollo                         82
pollo en polvo                         64
pollo troceado                         45
                                     ... 
pollo o ternera                         1
caldo de pollo o de cocido              1
un pollo                                1
caldo de pollo cantidad necesaria       1
pollo en filetes                        1
Name: ingredient, Length: 365, dtype: int64

In [26]:
filter_atun = recipe_df['atún'] > 86
recipe_df[filter_atun]["ingredient"].value_counts()

atún                        186
atún en aceite               62
atún en conserva             50
atún en agua                 32
atún al natural              13
                           ... 
atún o albacora               1
atún o bonito en rodajas      1
g. atún en lata               1
bonito o atún en trozos       1
atún entero                   1
Name: ingredient, Length: 71, dtype: int64

In [27]:
filter_garbanzos = recipe_df['garbanzos'] > 86
recipe_df[filter_garbanzos]["ingredient"].value_counts()

garbanzos                                                   179
garbanzos cocidos                                           116
harina de garbanzos                                          12
garbanzos cocidos                                            11
garbanzos                                                    10
garbanzo                                                      6
garbanzos secos                                               5
garbanzos remojados                                           5
garbanzos cocinados                                           5
gr garbanzos                                                  3
garbanzos.                                                    3
gr. garbanzos                                                 2
garbanzos remojados horas                                     2
tempeh de garbanzos                                           2
g garbanzos                                                   2
garbanzos previamente remojados         

In [28]:
filter_judías_verdes = recipe_df['judías verdes'] > 86
recipe_df[filter_judías_verdes]["ingredient"].value_counts()

jas verdes     156
jas verdes       5
verde            2
jas verdes.      2
verdes           2
verdes           1
uvas verdes      1
Name: ingredient, dtype: int64

In [29]:
filter_jamón = recipe_df['jamón'] > 86
recipe_df[filter_jamón]["ingredient"].value_counts()

jamón serrano                      200
jamón                              127
jamón cocido                       120
jamón york                          40
jamón dulce                         24
                                  ... 
jamón ibérico en trozo grande.       1
jamón cren cubos                     1
jamón ahumado en trozos grandes      1
jamón ibérico picado                 1
jamón serrano en un trozo            1
Name: ingredient, Length: 222, dtype: int64

In [30]:
filter_pan = recipe_df['pan'] > 86
recipe_df[filter_pan]["ingredient"].value_counts()

pan rallado                   517
pan                           270
panadería                      48
pan de molde                   46
pan molido                     45
                             ... 
pan rallado gr                  1
unidad pan pita                 1
panqueques rectangulares        1
¾ de pan rallado sazonado       1
miga de pan del día             1
Name: ingredient, Length: 393, dtype: int64

In [31]:
filter_pasta = recipe_df['pasta'] > 86
recipe_df[filter_pasta]["ingredient"].value_counts()

pasta                               59
pasta                               53
pasta brisa                         52
pasta de ajo.                       44
pasta corta                         43
                                    ..
pasta aproximadamente                1
ajo en pasta al gusto                1
pasta arcobaleno o pasta corta       1
pasta brisa un disco refrigerado     1
pasta de lima                        1
Name: ingredient, Length: 215, dtype: int64

In [32]:
filter_gluten = recipe_df["gluten_free"] == 1

recipe_df[filter_gluten]["ingredient"].value_counts().index.tolist()

['sal',
 'aceite de oliva',
 'huevo',
 'pimienta',
 'cebolla',
 'pimienta negra',
 'diente de ajo',
 'perejil',
 'azúcar',
 'zanahoria',
 'limón',
 'vino blanco',
 'dientes de ajo',
 'aceite',
 'pimiento rojo',
 'laurel',
 'tomillo',
 'tomate',
 'mantequilla',
 'pimiento verde',
 'cebolleta',
 'arroz',
 'patata',
 'calabacín',
 'patatas',
 'ajo',
 'zanahorias',
 'miel',
 'puerro',
 'pimentón dulce',
 'tomates',
 'romero',
 'nata líquida',
 'cebollino',
 'vinagre',
 'leche',
 'vinagre de jerez',
 'caldo de verduras',
 'orégano',
 'cebollas',
 'menta',
 'albahaca',
 'canela',
 'canela en polvo',
 'azúcar glas',
 'nata',
 'zumo de limón',
 'azúcar moreno',
 'berenjena',
 'espinacas',
 'pimentón',
 'cebolla morada',
 'pimienta negra molida',
 'azafrán',
 'vainilla',
 'tomates cereza',
 'perejil picado',
 'caldo de pescado',
 'harina de maíz',
 'limones',
 'jas verdes',
 'manzana',
 'tomate cereza',
 'pimientos rojos',
 'azúcar ',
 'maíz',
 'queso blanco',
 'champiñón',
 'mostaza',
 'guisan